In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [36]:
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv') 

In [37]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [38]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Merging two data frame

In [39]:
movies=movies.merge(credits, on='title')

In [40]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

**For a content-based movie recommender system using the attributes provided, you would typically consider the following attributes:
**
> 1. Genres: Utilize the genres of movies to recommend similar movies. You can calculate similarity between movies based on shared genres.
> 2. Keywords: Analyze keywords associated with each movie to find similarities between them. Keywords can indicate themes, plot elements, or other significant aspects of the movies.
> 3. Original Language: Consider the original language of the movie. Users might prefer movies in certain languages, so you can recommend movies in languages they prefer.
> 4. Overview: Analyze the overview or synopsis of each movie to understand its content. You can use natural language processing techniques to extract key information and find similarities between movie plots.
> 5. Popularity: Take into account the popularity of movies. You can recommend popular movies similar to those the user has already watched.
> 6. Runtime: Consider the runtime of movies. Users might have preferences regarding movie duration, so recommend movies with similar runtime.
> 7. Vote Average and Vote Count: Consider user ratings and the number of votes. Recommend movies with similar ratings or with a similar number of votes.
> 8. Cast and Crew: Analyze the cast and crew of each movie. If a user enjoys movies with specific actors or directors, you can recommend movies featuring those individuals.




In [41]:
# Define the features to keep
features_to_keep = ['movie_id','genres', 'keywords', 'original_language', 'overview', 
                    'popularity', 'runtime', 'vote_average', 'vote_count', 
                    'cast', 'crew','title']

# Filter the DataFrame to keep only the desired features
filtered_df = movies[features_to_keep]


In [42]:
filtered_df.isnull().sum()

movie_id             0
genres               0
keywords             0
original_language    0
overview             3
popularity           0
runtime              2
vote_average         0
vote_count           0
cast                 0
crew                 0
title                0
dtype: int64

In [43]:
# Remove rows with null values
filtered_df=filtered_df.dropna()

In [44]:
filtered_df.isnull().sum()

movie_id             0
genres               0
keywords             0
original_language    0
overview             0
popularity           0
runtime              0
vote_average         0
vote_count           0
cast                 0
crew                 0
title                0
dtype: int64

In [45]:
filtered_df.duplicated().sum()

0

In [46]:
filtered_df.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [47]:
import ast

class DataProcessor:
    def __init__(self, data):
        self.data = data
        
    def extract_fields(self, json_string):
        data_list = ast.literal_eval(json_string)
        return [data['name'] for data in data_list]
    def extract_top_actors(self,cast_json_string):
        cast_list = ast.literal_eval(cast_json_string)
        count =0
        top_actor = []
        for actor in cast_list:
            if(count!=3):
                top_actor.append(actor['name'])
                count+=1
            else:
                break
                
        
        return top_actor
    def extract_director(self,cast_json_string):
        d = ast.literal_eval(cast_json_string)
        l = []
        for i in d:
            if i['job'] =='Director':
                l.append(i['name'])
                break
        return l        
                
    def conver_string_to_list(self, string):
        return string.split()
    def remove_spaces_from_list(self,input_list):
    
        return [element.replace(" ", "") for element in input_list]
    def join_elements_with_spaces(self,input_list):
   
        return " ".join(input_list)
        
    def process_field(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.extract_fields)
    def process_cast(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.extract_top_actors)
    def process_crew(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.extract_director) 
    def process_overview(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.conver_string_to_list)
    def remove_space(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.remove_spaces_from_list)
    def join_list(self, field_name):
        self.data[field_name] = self.data[field_name].apply(self.join_elements_with_spaces)


        
        
        

In [48]:
processor = DataProcessor(filtered_df)

In [49]:
# Call the process_genres method to process the genres, keywords,cast column
processor.process_field('genres')
processor.process_field('keywords')
processor.process_cast('cast')
processor.process_crew('crew')
processor.process_overview('overview')

#Remove Spaces
processor.remove_space('genres')
processor.remove_space('crew')
processor.remove_space('keywords')
processor.remove_space('cast')


In [50]:
filtered_df.iloc[0].genres

['Action', 'Adventure', 'Fantasy', 'ScienceFiction']

In [51]:
filtered_df['tags'] = filtered_df['genres']+filtered_df['keywords']+filtered_df['crew']+filtered_df['cast']+filtered_df['overview']

In [52]:
filtered_df.head()

,movie_id,genres,keywords,original_language,overview,popularity,runtime,vote_average,vote_count,cast,crew,title,tags
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",en,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,162.0,7.2,11800,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",en,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,169.0,6.9,4500,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",en,"[A, cryptic, message, from, Bond’s, past, send...",107.376788,148.0,6.3,4466,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",en,"[Following, the, death, of, District, Attorney...",112.312950,165.0,7.6,9106,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",en,"[John, Carter, is, a, war-weary,, former, mili...",43.926995,132.0,6.1,2124,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."


In [53]:
new_df = filtered_df[['movie_id','title','tags']]
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."


In [54]:

processor1 = DataProcessor(new_df)
processor1.join_list('tags')


/tmp/ipykernel_33/2082211432.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[field_name] = self.data[field_name].apply(self.join_elements_with_spaces)


In [55]:
new_df['tags'][0]

'Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d JamesCameron SamWorthington ZoeSaldana SigourneyWeaver In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [56]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_33/3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [57]:
ps = PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)  
new_df['tags'] = new_df['tags'].apply(stem)

new_df['tags'][0]

/tmp/ipykernel_33/219814516.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d jamescameron samworthington zoesaldana sigourneyweav in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [58]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray()


In [59]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [61]:
similarity = cosine_similarity(vectors)


In [67]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)
        

In [68]:
recommend('Gandhi')

Gandhi, My Father
Guiana 1838
The Wind That Shakes the Barley
Mr. Turner
A Passage to India


In [73]:
import pickle
#pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))